In [3]:
sample_gcc = "gcc -c -o vm_map.o -include /home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/config.h -Wall -std=gnu89 -DKERNEL -include /home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/arch/arch-config.h -include /home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/arch/board-ia32_default-config.h  -m32 -W -Wall -MD  -I. -I/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32  -I/home/anton/Sandbox/PhantomOS/test/phantomuserland/include -I.. -I/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel -I/home/anton/Sandbox/PhantomOS/test/phantomuserland/include -I/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32 -nostdinc -DARCH_ia32=1 -DBOARD_ia32_default=1 -DBOARD=ia32_default -W -Wall -fno-strict-aliasing -ffreestanding -fno-inline -fkeep-inline-functions -fno-stack-protector -pipe -O0 -gstabs+  vm_map.c"

In [4]:
import re

def generate_cflow_command(file_name, gcc_command):

    cc_options = re.findall(r"-I\S*|-include \S*|-D\S*|-imacros \S*|-nostdinc", gcc_command)
#     print(cc_options)

    # file_name = gcc_command.split(" ")[-1]
    # print(file_name)

    cc_command = "gcc -E " + " ".join(cc_options)
    # print(cc_command)
    
    cflow_command = 'cflow -i +_s --brief --cpp="{}" {}'.format(cc_command, file_name)
    
    return cflow_command
    
# print(generate_cflow_command(sample_gcc))

In [5]:
text = None

with open("build_kernel.log", "r") as f:
    text = f.read()


In [6]:
matches = re.findall(r"\!source_file\{(\S*)\}.*?\n(.*?)\n", text)

files_to_cflow_commands = {}

for m in matches:
    file_name = m[0]
    gcc_command = m[1]
    cflow_command = generate_cflow_command(file_name, gcc_command)

    if file_name in files_to_cflow_commands.keys():
        print("Warning: duplicating files: {}".format(file_name))
    else:
        files_to_cflow_commands[file_name] = cflow_command



In [7]:
import subprocess

cflow_results = {}

for file_name, cmd in files_to_cflow_commands.items():
    res = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True, check=True)
    cflow_results[file_name] = res.stdout

# [s for s in cflow_commands if "physalloc" in s][0]

In [8]:
len(cflow_results)

858

In [9]:
# sample_res_text = """
# paint_allocator_memory_map() <void paint_allocator_memory_map (window_handle_t w, rect_t *r, physalloc_t *m) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:341>:
#     w_draw_pixel()
#     calc_pixel_color() <rgba_t calc_pixel_color (physalloc_t *m, int elem, int units_per_pixel) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:360>:
#         __builtin_popcount()
# phantom_phys_alloc_init() <void phantom_phys_alloc_init (physalloc_t *arena, u_int32_t n_alloc_units) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:74>:
#     malloc()
#     do_phantom_phys_alloc_init() <void do_phantom_phys_alloc_init (physalloc_t *arena, u_int32_t n_alloc_units, int n_map_elems, void *mapbuf) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:44>:
#         hal_spin_init()
#         hal_save_cli()
#         hal_spin_lock()
#         memset()
#         hal_spin_unlock()
#         hal_sti()
# phantom_phys_alloc_init_static() <void phantom_phys_alloc_init_static (physalloc_t *arena, u_int32_t n_alloc_units, void *mapbuf) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:67>:
#     do_phantom_phys_alloc_init() <void do_phantom_phys_alloc_init (physalloc_t *arena, u_int32_t n_alloc_units, int n_map_elems, void *mapbuf) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:44>: [see 7]
# phantom_phys_alloc_page() <errno_t phantom_phys_alloc_page (physalloc_t *arena, physalloc_item_t *ret) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:82>:
#     panic()
#     hal_spin_lock_cli()
#     hal_spin_unlock_sti()
# phantom_phys_alloc_region() <errno_t phantom_phys_alloc_region (physalloc_t *arena, physalloc_item_t *ret, size_t npages) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:214>:
#     panic()
#     hal_spin_lock_cli()
#     hal_spin_unlock_sti()
#     memset()
# phantom_phys_free_page() <void phantom_phys_free_page (physalloc_t *arena, physalloc_item_t free) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:136>:
#     panic()
# phantom_phys_free_region() <void phantom_phys_free_region (physalloc_t *arena, physalloc_item_t start, size_t _n_pages) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:162>:
#     panic()
# phantom_phys_stat_arena() <long phantom_phys_stat_arena (physalloc_t *arena) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/physalloc.c:280>:
#     panic()
#     umax() <__inline u_int umax (u_int a, u_int b) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/include/sys/libkern.h:42>
# """

In [10]:
from queue import Queue

class FuncNode:
    def __init__(self, name, sign, location):
        self.name = name
        self.sign = sign
        self.location = location
    
    def compare(self, other):
        if not isinstance(other, FuncNode):
            return False
        else:
            return self.name == other.name and self.sign == other.sign and self.location == other.location
    
    def is_complete(self):
        return (self.sign is not None) and (self.location is not None)
    

class Index:
    
    def __init__(self):
        self.store = {}
        self.relations = {}
        self.reverse_relations = {}
    
    def update_store(self, new_node):
        # If new absent info about node, update
        # otherwise return or give warning
        
        if (not new_node.name in self.store.keys()):
            self.store[new_node.name] = new_node
            
            # also update graphs with empty nodes
            self.relations[new_node.name] = set()
            self.reverse_relations[new_node.name] = set()
            
        else:
            found_node = self.store[new_node.name]
            if (new_node.is_complete() and found_node.is_complete()):
                if not found_node.compare(new_node):
                    print("WARNING : duplicating def of {}".format(new_node.name))
                    print("          old: {} at {}".format(found_node.sign, found_node.location))
                    print("          new: {} at {}".format(new_node.sign, new_node.location))
            if (new_node.is_complete() and not self.store[new_node.name].is_complete):
                self.store[new_node.name].sign = new_node.sign
                self.store[new_node.name].location = new_node.location
    
    def get_callers(self, name):
        return list(self.reverse_relations[name])
    
    def get_callees(self, name):
        return list(self.relations[name])
    
    def get_store(self):
        return self.store
    
    def _bfs(self, graph, start_nodes, term_condition, ignore_nodes):
        
        visited = set()
        
        subgraph = {}
        
        to_visit = Queue()
        
        for node in start_nodes:
            to_visit.put(node)
            visited.add(node)
        
        while (not to_visit.empty()):
            curr = to_visit.get()
            
            subgraph[curr] = set()
            
            if term_condition(curr):
                continue
            
            for next_func in graph[curr]:
                
                if next_func in ignore_nodes:
                    continue
                
                subgraph[curr].add(next_func)
                
                if not next_func in visited:
                    to_visit.put(next_func)
                    visited.add(next_func)
        
        # Just sanity check
        
        assert(len(subgraph.keys()) == len(visited))
        
        return subgraph
            
    
    def get_subgraph(self, starting_nodes, term_condition, ignore_nodes):
        return self._bfs(self.relations, starting_nodes, term_condition, ignore_nodes)
    
    def add(self, from_name, to_node):
        # from_name = from_node.name
        to_name = to_node.name
        
        # self.update_store(from_node)
        self.update_store(to_node)
        
        # caller->calee
        if (from_name in self.relations.keys()):
            self.relations[from_name].add(to_name)
        else:
            self.relations[from_name] = set([to_name])
        
        # calee->caller
        if (to_name in self.reverse_relations.keys()):
            self.reverse_relations[to_name].add(from_name)
        else:
            self.reverse_relations[to_name] = set([from_name])
            
    
    def debug_print(self):
        print("Store: ")
        for k, v in self.store.items():
            print("- [{}] {} {} {}".format(k, v.name, v.sign, v.location))
        print("Relations: ")
        for k, v in self.relations.items():
            print("- {}".format(k))
            for to_name in v:
                print("    - {}".format(to_name))


def parse_cflow_output(index, res_text):
    
    stack = []
    ident = 0
    delim = "    "
    prev_name = None
    
    for l in res_text.split("\n"):
        curr = l
        curr_ident = 0
        while (curr.startswith(delim)):
            curr_ident += 1
            curr = curr[len(delim):]
        
        func_name = None
        func_sig = None
        func_location = None
        
        curr = curr.strip()
        res = re.findall(r"^(\S+) \<(.*?) at (.*?)\>|^(\S+)", curr)
        if len(res) != 1:
            # print("Warning, skipping iteration with input '{}'".format(curr))
            continue
            
        res = res[0]
        if res[3]:
                # print("SYM: {}".format(res[3]))
                func_name = res[3]
        else:
                # print("DEC: {}, {}, {}".format(res[0], res[1], res[2]))
                func_name = res[0]
                func_sig = res[1]
                func_location = res[2]

        
        new_node = FuncNode(func_name, func_sig, func_location)
        
        if curr_ident > ident:
            stack.append(prev_name)
            ident = curr_ident
        
        if curr_ident < ident:
            stack.pop()
            ident = curr_ident
        
        if len(stack) > 0:
            index.add(stack[-1], new_node)
        else:
            index.update_store(new_node)
            
        prev_name = func_name
            
        
        

In [11]:
# Misc functions

def _dfs(graph, visited, start_node, depth):
    res = set([(depth, start_node)])
    
    if depth == 0:
        return res
    
    for v in graph[start_node]:
        if not v in visited:
            visited.add(v)
            res.update(_dfs(graph, visited, v, depth - 1))
    
    return res

def get_neighbours(graph, start_node, depth):
    visited = set()
    res = {}
    raw_res = _dfs(graph, visited, start_node, depth)
    
    for curr_d, node in raw_res:
        curr_d = depth - curr_d
        if not curr_d in res.keys():
            res[curr_d] = set()
        res[curr_d].add(node)
    
    return res
        

In [12]:
# index = Index()

# parse_cflow_output(index, sample_res_text)

# index.debug_print()

In [13]:
kernel_files = [f for f in cflow_results.keys() if "kernel" in f]

# Avoiding duplicating definitions
excluded_files = [
    "/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/floppy.c",
    "/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/unix/phantom.c",
    "/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/init.h", # unix hal > this header
]


In [14]:
vm_files = [f for f in cflow_results.keys() if "/vm/" in f]

In [15]:
# del index

kernel_index = Index()

for f in kernel_files:
    if f in excluded_files:
        continue
        
    t = cflow_results[f]
    parse_cflow_output(kernel_index, t.decode("utf-8"))


          old: FRESULT dir_read (DIR *dj) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ff.c:980
          new: size_t dir_read (struct uufile *f, void *dest, size_t bytes) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/unix/dir.c:270


In [16]:
vm_index = Index()

for f in vm_files:
    if f in excluded_files:
        continue
        
    t = cflow_results[f]
    parse_cflow_output(vm_index, t.decode("utf-8"))

          old: void hal_cpu_reset_real (void) at /home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/init.h:128
          new: void hal_cpu_reset_real () at /home/anton/Sandbox/PhantomOS/test/phantomuserland/phantom/vm/unix_hal.c:408


In [17]:
# Sanity check
# If something uses physalloc directly else then hal

# phantom_phys_alloc_page
kernel_index.get_callers("phantom_phys_alloc_page()")

['alloc_ldt_selector()',
 'phantom_phys_alloc_init()',
 'hal_alloc_phys_page()',
 'hal_alloc_vaddress()']

In [18]:
kernel_index.get_callers("hal_alloc_phys_page()")

['page_fault_read()',
 'driver_virtio_net_write()',
 'provide_buffers()',
 'page_fault_snap_aid()',
 'page_fault_write()']

In [19]:
kernel_index.get_callees("hal_alloc_phys_page()")

['phantom_phys_alloc_page()', 'panic()', 'physmem_try_to_reclaim_page()']

In [20]:
# Let's try to find kernel functions used by vm

In [21]:
vm_kernel_intersection = set(vm_index.get_store().keys()).intersection(set(kernel_index.get_store().keys()))

print(len(vm_kernel_intersection))

# There are 3 types of functions in this intersection
# That are implemented in kernel and called by vm
# That are implemented in kernel and vm and called by vm
# That are implemented in kernel and vm and called by kernel and vm
# Probably also that are implemented in vm and called by kernel

# Using this call graph we can't be 100% sure that we have only delcaration but not implementation and vice versa
# So, let's try to derive two groups of functions: called only by vm and called both by vm and kernel. 
# Also let's check if there functions called by kernel, but not vm in this intersection

vm_kernel_calls = {
    "vm" : set(),
    "kernel" : set(),
    "both" : set(),
}

for f_name in sorted(list(vm_kernel_intersection)):
    
    callers_vm = []
    callers_kernel = []
    
    try:
        callers_vm = vm_index.get_callers(f_name)
    except Exception:
        callers_vm = []
        
    try:
        callers_kernel = kernel_index.get_callers(f_name)
    except Exception:
        callers_kernel = []
    
    if (len(callers_vm) > 0 and len(callers_kernel) > 0):
        vm_kernel_calls["both"].add(f_name)
    elif len(callers_vm) > 0:
        vm_kernel_calls["vm"].add(f_name)
    elif len(callers_kernel) > 0:
        vm_kernel_calls["kernel"].add(f_name)
    

print("called only by kernel:")
[print("    - {}".format(f_name)) for f_name in sorted(list(vm_kernel_calls["kernel"]))]

print("called only by vm:")
[print("    - {}".format(f_name)) for f_name in sorted(list(vm_kernel_calls["vm"]))]

print("called by both:")
[print("    - {}".format(f_name)) for f_name in sorted(list(vm_kernel_calls["both"]))]

print("Done.")

144
called only by kernel:
    - activate_all_threads()
    - bulk_read_f()
    - bulk_seek_f()
    - check_global_lock_entry_count()
    - paint_object_memory_map()
called only by vm:
    - getDebugChar()
    - hal_halt()
    - phantom_connect_object()
    - phantom_connect_object_internal()
    - phantom_set_console_getchar()
    - putDebugChar()
    - unwire_page_for_addr()
    - vm_syscall_block()
called by both:
    - __builtin_offsetof()
    - __builtin_va_end()
    - __builtin_va_start()
    - __sync_add_and_fetch()
    - __sync_lock_test_and_set()
    - atol()
    - calloc()
    - dbg_add_command()
    - dbg_init()
    - drv_video_get_default_mouse_bmp()
    - drv_video_init_windows()
    - drv_video_window_create()
    - exit()
    - fast_time()
    - free()
    - gdb_stub_handle_cmds()
    - getchar()
    - hal_cli()
    - hal_cond_broadcast()
    - hal_cond_init()
    - hal_cond_signal()
    - hal_cond_wait()
    - hal_cpu_reset_real()
    - hal_disable_preemption()
    - ha

In [22]:
# What we see is that most of calls are called by both
# We can divide these calls into 5 groups
# 1. Phantom's HAL
# 2. Direct Phantom kernel calls
# 3. Libc functions
# 4. Windows subsystem (How? :D)
# 5. PVM calls

# Now, need to understand what is non-persistent and can be implemented in Genode 
# and what should be ported from kernel.
# Then, we can try to find what those calls use inside the kernel and what exact files we need
# Also, it worth testing 


In [23]:
pvm_memory_calls = [
    "vm_lock_persistent_memory()",
    "vm_map_page_mark_unused()",
    "vm_unlock_persistent_memory()",
    "vm_syscall_block()",
    "phantom_connect_object()",
    "phantom_connect_object_internal()",
    "phantom_disconnect_object()",
    "wire_page_for_addr()",
    "unwire_page_for_addr()",
    "ref_dec_o()",
    "ref_inc_o()",
    "ref_inc_p()",
    "pvm_add_object_to_restart_list()",
    "pvm_alloc_init()",
    "pvm_alloc_threaded_init()",
    "pvm_bulk_init()",
    "pvm_create_binary_object()",
    "pvm_create_int_object()",
    "pvm_create_null_object()",
    "pvm_create_object()",
    "pvm_create_string_object()",
    "pvm_create_string_object_binary()",
    "pvm_exec_lookup_class_by_name()",
    "pvm_exec_run_method()",
    "pvm_get_int_class()",
    "pvm_get_method_name()",
    "pvm_get_string_class()",
    "pvm_object_dump()",
    "pvm_object_is_allocated_assert()",
    "pvm_object_is_allocated_light()",
    "pvm_remove_object_from_restart_list()",
    "pvm_root_init()",
    "load_code()",
]

In [24]:
# Trying to get interfaces we need to implement
# We are ok to implement really low level disk ops (write/read), so leaf functions of them will be ok
# However, we want to substitute memory interfaces, so termination condition should be corresponding
# Also, we don't want any std calls

std_calls = [ # stdlib calls to avoid
    '__builtin_popcount()',
    '__builtin_va_end()',
    '__builtin_va_start()',
    '__volatile()',
    'atomic_add()',
    'bzero()',
    'calloc()',
    'lprintf()',
    'memcpy()',
    'memset()',
    'printf()',
    'snprintf()',
    'strchr()',
    'vprintf()',
    'syslog()',
    'panic()',
    
    
    'malloc()',
    'free()'
]

kernel_mem_subgraph = kernel_index.get_subgraph(pvm_memory_calls, lambda func_name : func_name.startswith("phantom_phys_"), std_calls)

leaf_nodes = set([node for node in kernel_mem_subgraph.keys() if len(kernel_mem_subgraph[node]) == 0])
print("leaf nodes count: {}".format(len(leaf_nodes)))
print("total nodes count: {}".format(len(kernel_mem_subgraph.keys())))
leaf_nodes


leaf nodes count: 110
total nodes count: 284


{'GET_CPU_ID()',
 'amap_destroy()',
 'amap_init()',
 'amap_iterate_flags()',
 'amap_modify()',
 'arch_get_fault_address()',
 'arch_get_frame_pointer()',
 'arch_is_object_land_access_enabled()',
 'calc_pixel_color()',
 'dbg_add_command()',
 'disk_dequeue()',
 'disk_enqueue()',
 'disk_page_cache_modified()',
 'disk_page_io_seek()',
 'disk_raise_priority()',
 'dump_mem_stat()',
 'ftlbentry()',
 'get_current_thread()',
 'get_ss()',
 'hal_cond_broadcast()',
 'hal_cond_init()',
 'hal_cond_wait()',
 'hal_exit_kernel_thread()',
 'hal_init_object_vmem()',
 'hal_is_sti()',
 'hal_mem_pagesize()',
 'hal_mutex_init()',
 'hal_mutex_is_locked()',
 'hal_mutex_lock()',
 'hal_mutex_unlock()',
 'hal_object_space_address()',
 'hal_printf()',
 'hal_save_cli()',
 'hal_sem_init()',
 'hal_sleep_msec()',
 'hal_spin_init()',
 'hal_start_kernel_thread()',
 'hal_sti()',
 'is_on_reclaim_q()',
 'need_pageout()',
 'page_touch_history()',
 'page_touch_history_arg()',
 'pager_long_fsck()',
 'pager_superblock_ptr()',
 

In [25]:
leaf_nodes.intersection(set(pvm_memory_calls))

{'phantom_connect_object_internal()',
 'phantom_disconnect_object()',
 'pvm_add_object_to_restart_list()',
 'pvm_alloc_init()',
 'pvm_alloc_threaded_init()',
 'pvm_bulk_init()',
 'pvm_create_binary_object()',
 'pvm_create_int_object()',
 'pvm_create_null_object()',
 'pvm_create_object()',
 'pvm_create_string_object()',
 'pvm_create_string_object_binary()',
 'pvm_exec_lookup_class_by_name()',
 'pvm_exec_run_method()',
 'pvm_get_int_class()',
 'pvm_get_method_name()',
 'pvm_get_string_class()',
 'pvm_object_dump()',
 'pvm_object_is_allocated_assert()',
 'pvm_object_is_allocated_light()',
 'pvm_remove_object_from_restart_list()',
 'pvm_root_init()',
 'ref_dec_o()',
 'ref_inc_o()',
 'ref_inc_p()',
 'vm_map_page_mark_unused()'}

In [26]:
kernel_index.get_callees("pvm_alloc_init()")

[]

In [27]:
vm_index.get_callees("pvm_alloc_init()")

['pvm_collapse_free()',
 'pvm_alloc_threaded_init()',
 'pvm_alloc_is_free_object()',
 'panic()',
 'init_arenas()',
 'pvm_object_alloc()',
 'pvm_next_object()']

In [28]:
kernel_index.get_callers("pvm_alloc_init()")

['hal_init()']

In [29]:
[kernel_index.get_callers(f) for f in leaf_nodes.intersection(set(pvm_memory_calls))]

[['phantom_connection_callback_int()',
  'cn_udp_blocking_syscall_worker()',
  'ko_make_int()',
  'unlock_and_ret_int()',
  'cn_fio_blocking_syscall_worker()'],
 ['vm_verify_object()'],
 ['load_classes_module()'],
 ['ko_get_class()'],
 [],
 ['run_cb_thread()'],
 ['vm_syscall_block()',
  'cn_udp_blocking_syscall_worker()',
  'cn_timer_do_operation()',
  'unlock_and_ret_err()',
  'cn_fio_blocking_syscall_worker()'],
 ['ko_make_string_binary()', 'cn_fio_blocking_syscall_worker()'],
 ['main()'],
 ['cn_fio_blocking_syscall_worker()'],
 ['object2handle()'],
 ['hal_init()'],
 ['pool_el_destroy()'],
 ['cn_fio_blocking_syscall_worker()'],
 ['main()'],
 ['vm_map_page_fault_handler()'],
 ['vm_verify_object()'],
 ['vm_lock_persistent_memory()', 'run_cb()', 'ko_get_class()'],
 ['phantom_connect_object()', 'phantom_connection_callback_binary()'],
 ['object2handle()'],
 ['phantom_connection_callback_string()',
  'vm_syscall_block()',
  'cn_url_blocking_syscall_worker()',
  'ko_get_class()',
  'ko_mak

In [30]:
leaf_nodes - (set(pvm_memory_calls))

{'GET_CPU_ID()',
 'amap_destroy()',
 'amap_init()',
 'amap_iterate_flags()',
 'amap_modify()',
 'arch_get_fault_address()',
 'arch_get_frame_pointer()',
 'arch_is_object_land_access_enabled()',
 'calc_pixel_color()',
 'dbg_add_command()',
 'disk_dequeue()',
 'disk_enqueue()',
 'disk_page_cache_modified()',
 'disk_page_io_seek()',
 'disk_raise_priority()',
 'dump_mem_stat()',
 'ftlbentry()',
 'get_current_thread()',
 'get_ss()',
 'hal_cond_broadcast()',
 'hal_cond_init()',
 'hal_cond_wait()',
 'hal_exit_kernel_thread()',
 'hal_init_object_vmem()',
 'hal_is_sti()',
 'hal_mem_pagesize()',
 'hal_mutex_init()',
 'hal_mutex_is_locked()',
 'hal_mutex_lock()',
 'hal_mutex_unlock()',
 'hal_object_space_address()',
 'hal_printf()',
 'hal_save_cli()',
 'hal_sem_init()',
 'hal_sleep_msec()',
 'hal_spin_init()',
 'hal_start_kernel_thread()',
 'hal_sti()',
 'is_on_reclaim_q()',
 'need_pageout()',
 'page_touch_history()',
 'page_touch_history_arg()',
 'pager_long_fsck()',
 'pager_superblock_ptr()',
 

In [31]:
# Let's check which of those functions have actual implementation

sorted([f for f in leaf_nodes - (set(pvm_memory_calls)) if kernel_index.store[f].is_complete()])

['GET_CPU_ID()',
 'calc_pixel_color()',
 'disk_dequeue()',
 'disk_enqueue()',
 'disk_page_cache_modified()',
 'disk_page_io_seek()',
 'disk_raise_priority()',
 'dump_mem_stat()',
 'get_current_thread()',
 'get_ss()',
 'hal_exit_kernel_thread()',
 'hal_init_object_vmem()',
 'hal_mem_pagesize()',
 'hal_object_space_address()',
 'hal_printf()',
 'is_on_reclaim_q()',
 'need_pageout()',
 'page_touch_history()',
 'page_touch_history_arg()',
 'pager_long_fsck()',
 'phantom_fsck_super()',
 'phantom_is_a_real_kernel()',
 'phantom_phys_free_count()',
 'rdr6()',
 'trap_ignore()',
 'unwire_page()',
 'visit_all()',
 'vm_map_get_object_address_space_start()',
 'vm_verify_vm()',
 'wire_page()']

In [32]:
# Let's check which of those functions have no actual implementation

sorted([f for f in leaf_nodes - (set(pvm_memory_calls)) if not kernel_index.store[f].is_complete()])

['amap_destroy()',
 'amap_init()',
 'amap_iterate_flags()',
 'amap_modify()',
 'arch_get_fault_address()',
 'arch_get_frame_pointer()',
 'arch_is_object_land_access_enabled()',
 'dbg_add_command()',
 'ftlbentry()',
 'hal_cond_broadcast()',
 'hal_cond_init()',
 'hal_cond_wait()',
 'hal_is_sti()',
 'hal_mutex_init()',
 'hal_mutex_is_locked()',
 'hal_mutex_lock()',
 'hal_mutex_unlock()',
 'hal_save_cli()',
 'hal_sem_init()',
 'hal_sleep_msec()',
 'hal_spin_init()',
 'hal_start_kernel_thread()',
 'hal_sti()',
 'pager_superblock_ptr()',
 'pahantom_free_phys_mem_kb()',
 'pahantom_total_phys_mem_kb()',
 'phantom_calc_sb_checksum()',
 'phantom_map_page()',
 'phantom_phys_alloc_init()',
 'phantom_phys_alloc_init_static()',
 'phantom_phys_alloc_page()',
 'phantom_phys_alloc_region()',
 'phantom_phys_free_region()',
 'phantom_symtab_getname()',
 'phantom_unmap_page()',
 'stack_dump_from()',
 'superblocks_are_equal()',
 't_current_get_priority()',
 't_current_set_name()',
 't_current_set_priority(

In [33]:
# print(len(kernel_index.store))
# print(len(kernel_index.relations.keys()))
# # [f for f in kernel_index.relations.keys() if len(kernel_index.relations[f]) == 0]
# print(set(kernel_index.store.keys()) - set(kernel_index.relations.keys()))

In [34]:
# This time stop on pager as well
# kernel_mem_subgraph = kernel_index.get_subgraph(pvm_memory_calls, lambda func_name : func_name.startswith("hal_"))

# leaf_nodes = set([node for node in kernel_mem_subgraph.keys() if len(kernel_mem_subgraph[node]) == 0])
# print(len(leaf_nodes))
# leaf_nodes

In [35]:
# kernel_index.get_callees("disk_enqueue()")
# kernel_index.get_callers("raw_alloc()")

In [36]:
# Getting list of files based on subgraph

locations_set = [kernel_index.store[f_name].location for f_name in kernel_mem_subgraph.keys() if f_name in kernel_index.store.keys()]

files_set = set([loc.split(":")[0] for loc in locations_set if loc])

files_set

{'/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/hal.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/proc_reg.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/smp.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/pager_io_req.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/threads.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/fsck.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal_physmem.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32/paging.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32/spinlock.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32/traps.c',
 '

In [37]:
# We have some ia32 files. What was used from them?

ia32_func_names = [(f_name, kernel_index.store[f_name].location) for f_name in kernel_mem_subgraph.keys() if f_name in kernel_index.store.keys()]
ia32_func_names = [x for x in ia32_func_names if x[1] and "ia32" in x[1]]
ia32_func_names

[('trap_ignore()',
  '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32/traps.c:95'),
 ('rdr6()',
  '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/proc_reg.h:266'),
 ('get_ss()',
  '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/ia32/proc_reg.h:177'),
 ('hal_page_control_etc()',
  '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32/paging.c:30'),
 ('spin_dump()',
  '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/ia32/spinlock.c:37')]

In [38]:
# Nothing really crucial except hal_page_control. Generating list of files

files_list_filtered = [f for f in files_set if not "ia32" in f]

files_list_filtered

['/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/hal.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/threads.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/pager_io_req.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/snap_sync.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/include/kernel/smp.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/pagelist.h',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/fsck.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/pager.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/disk.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/user_trap.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/hal.c',
 '/home/anton/Sandbox/PhantomOS/test/phantomuserland/oldtree/kernel/phantom/vm_connect.c',
 '/h

In [39]:
# Removing common prefix and trying to generate make rules

files_list_filtered = sorted([f[len("/home/anton/Sandbox/PhantomOS/test/phantomuserland/"):] for f in files_list_filtered])

files_list_filtered

['include/hal.h',
 'include/kernel/smp.h',
 'include/pager_io_req.h',
 'include/threads.h',
 'oldtree/kernel/phantom/disk.c',
 'oldtree/kernel/phantom/fsck.c',
 'oldtree/kernel/phantom/hal.c',
 'oldtree/kernel/phantom/hal_physmem.c',
 'oldtree/kernel/phantom/pagelist.h',
 'oldtree/kernel/phantom/pager.c',
 'oldtree/kernel/phantom/physalloc.c',
 'oldtree/kernel/phantom/snap_sync.c',
 'oldtree/kernel/phantom/user_trap.c',
 'oldtree/kernel/phantom/vm_connect.c',
 'oldtree/kernel/phantom/vm_load.c',
 'oldtree/kernel/phantom/vm_map.c']

In [40]:
# What about init functions

init_subgraph = kernel_index.get_subgraph(["main()"], lambda func_name : False, [])

init_subgraph

{'main()': {'InitializeFullAcpi()',
  'activate_all_threads()',
  'arch_threads_init()',
  'board_init_kernel_timer()',
  'board_start_smp()',
  'connect_ide_io()',
  'dbg_init()',
  'dpc_init()',
  'exit()',
  'hal_cpu_reset_real()',
  'hal_init()',
  'hal_init_physmem_alloc_thread()',
  'hal_object_space_address()',
  'hal_page_control()',
  'hal_set_softirq_handler()',
  'hal_sleep_msec()',
  'heap_init_mutex()',
  'init_irq_allocator()',
  'init_main_event_q()',
  'load_classes_module()',
  'net_stack_init()',
  'net_timer_init()',
  'phantom_check_disk_check_virtmem()',
  'phantom_find_drivers()',
  'phantom_init_part_pool()',
  'phantom_init_vesa()',
  'phantom_init_vm86()',
  'phantom_paging_init()',
  'phantom_port_init()',
  'phantom_process_boot_options()',
  'phantom_request_timed_call()',
  'phantom_shutdown()',
  'phantom_start_boot_modules()',
  'phantom_start_video_driver()',
  'phantom_threads_init()',
  'phantom_timed_call_init()',
  'phantom_unix_fs_init()',
  'phanto

In [41]:
get_neighbours(init_subgraph, "main()", 3)

{3: {'AcpiException()',
  'AcpiInstallFixedEventHandler()',
  'AcpiInstallNotifyHandler()',
  'AcpiWalkNamespace()',
  'MD5_Final()',
  'MD5_Init()',
  'MD5_Update()',
  'NotifyHandler()',
  'XTeaCryptStr()',
  'XTeaDecryptStr()',
  '__builtin_constant_p()',
  '__cfree()',
  '__inw()',
  '__inwc()',
  '__outw()',
  '__outwc()',
  '_create()',
  '_create_free()',
  '_ff()',
  '_ff_fail()',
  '_show_free()',
  '_wtty_test()',
  'acpi_power_off()',
  'allocate_cbuf_mem()',
  'amap_check()',
  'amap_check_modify()',
  'amap_destroy()',
  'amap_init()',
  'amap_iterate_flags()',
  'arch_get_rtc_delta()',
  'arp_cache_compare()',
  'arp_cache_hash()',
  'arp_cleanup_thread()',
  'arp_input()',
  'arp_insert()',
  'arp_lookup()',
  'arp_retransmit_thread()',
  'board_make_driver_map()',
  'bzero()',
  'calloc()',
  'cbuf_free_chain()',
  'cbuf_free_chain_noblock()',
  'cbuf_get_chain()',
  'cbuf_get_len()',
  'cbuf_memcpy_to_chain()',
  'cbuf_merge_chains()',
  'count()',
  'create_pool()',
 

In [42]:
kernel_index.get_callers("hal_page_control()")

['memcpy_p2v()',
 'hal_copy_page_v2p()',
 'mark_for_snap()',
 'page_clear_engine_clear_page()',
 'page_fault_read()',
 'physmem_try_to_reclaim_page()',
 'vm_page_req_pageout()',
 'pagein_callback()',
 'get_fb_seg()',
 'hal_pages_control()',
 'page_fault_snap_aid()',
 'main()',
 'page_fault_write()',
 'memzero_page_v2p()',
 'memcpy_v2p()']

In [43]:
kernel_index.get_callers("fsck_create_map()")

['phantom_fsck_rebuild_free()', 'phantom_free_snap()']

In [44]:
kernel_index.get_callers("disk_enqueue()")

['pager_enqueue_for_pagein()', 'pager_enqueue_for_pageout()']

In [ ]:
# Let's try to import a new init routine and get all functions that it calls. Then try to build dependencies using kernel index

